In [1]:
import pandas as pd


In [2]:
housing=pd.read_csv("HOUSINGDATA.csv")

In [3]:
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [6]:
housing=strat_train_set.copy()

In [7]:
housing = strat_train_set.drop("MEDV", axis=1)
housing_labels = strat_train_set["MEDV"].copy()

In [8]:
median = housing["RM"].median()
housing["RM"].fillna(median)


254    6.108
348    6.635
476    6.484
321    6.376
326    6.312
       ...  
155    6.152
423    6.103
98     7.820
455    6.525
216    5.888
Name: RM, Length: 404, dtype: float64

In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = "median")
imputer.fit(housing)

SimpleImputer(strategy='median')

In [10]:
imputer.statistics_

array([2.86735e-01, 0.00000e+00, 9.90000e+00, 0.00000e+00, 5.38000e-01,
       6.21000e+00, 7.82000e+01, 3.12220e+00, 5.00000e+00, 3.37000e+02,
       1.90000e+01, 3.90955e+02, 1.15700e+01])

In [11]:
X = imputer.transform(housing)

In [12]:
housing_tr = pd.DataFrame(X,columns=housing.columns)

In [13]:
housing_tr.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.279908,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.712983,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.878750,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.210000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630250,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

In [15]:
housing_num_tr = my_pipeline.fit_transform(housing)

In [16]:
from sklearn.ensemble import RandomForestRegressor
  
model = RandomForestRegressor()
model.fit(housing_num_tr, housing_labels)

RandomForestRegressor()

In [17]:
some_data=housing.iloc[:5]
some_labels=housing_labels.iloc[:5]
prepared_data=my_pipeline.transform(some_data)


In [18]:
model.predict(prepared_data)


array([22.396, 25.547, 16.458, 23.342, 23.409])

In [19]:

list(some_labels)

[21.9, 24.5, 16.7, 23.1, 23.0]

In [20]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, housing_num_tr, housing_labels, scoring="neg_mean_squared_error",cv=10)
rmse_scores = np.sqrt(-scores)

In [21]:
rmse_scores


array([2.7749595 , 2.50561476, 4.45739856, 2.39381193, 3.28810186,
       2.61876848, 5.21031336, 3.28253057, 3.25483531, 3.33704453])

In [22]:
def print_scores(score):
    print("Scores:", score)
    print("Mean:", score.mean())
    print("Standard deviations:", score.std())



In [23]:
 print_scores(rmse_scores)

Scores: [2.7749595  2.50561476 4.45739856 2.39381193 3.28810186 2.61876848
 5.21031336 3.28253057 3.25483531 3.33704453]
Mean: 3.3123378857113943
Standard deviations: 0.8475360660880402


In [24]:
from joblib import dump, load
dump(model, 'DragonData.joblib')

['DragonData.joblib']

In [25]:
X_test=strat_test_set.drop("MEDV", axis=1)
Y_test=strat_test_set["MEDV"].copy()


In [26]:
import numpy as np
from sklearn.metrics import mean_squared_error
X_test_prepared=my_pipeline.transform(X_test)
final_predictions=model.predict(X_test_prepared)
final_mse=mean_squared_error(Y_test, final_predictions)
final_rmse=np.sqrt(final_mse)

In [27]:
final_rmse

2.9527822622461155

In [28]:
print(final_predictions,list(Y_test))

[25.058 11.263 25.718 21.813 18.863 15.199 20.881 14.216 31.807 41.743
 19.762 11.939 23.209 27.15  19.362 11.25  31.037 14.167 23.551 18.121
 19.424 17.888 17.776 21.975 18.865 31.381 15.93  33.004  8.685 33.331
 24.053 21.542 22.952 10.565 20.83  11.25  43.604 24.503 23.237 42.774
 24.23  30.35  19.929 20.872 18.873 32.582 44.395 19.828 20.24  21.583
 21.303 14.086 21.598 14.836 25.602 32.236 42.503 28.48  19.841 20.472
 46.773  9.985 19.081 24.728 14.94  33.962 19.537 18.054 19.371 34.376
 25.926 22.879 21.13  22.776 34.277 13.05  16.381 20.069 20.678 21.026
 22.507 21.036 14.317 23.243 20.475 21.298 14.086 21.423 22.135 23.184
 18.798 27.251  7.372 26.291 19.531 28.712 20.113 31.345 14.514 26.2
 21.305 20.431] [16.5, 10.2, 30.1, 23.0, 14.4, 15.6, 19.4, 14.1, 30.3, 35.2, 23.1, 13.8, 25.0, 27.9, 19.5, 12.3, 32.2, 13.5, 23.8, 21.7, 19.2, 19.5, 10.4, 23.2, 18.6, 28.5, 15.2, 32.0, 7.2, 34.6, 20.1, 20.6, 23.6, 13.1, 23.8, 12.7, 43.1, 24.7, 22.2, 44.0, 28.1, 31.0, 21.7, 23.4, 19.5, 33.1, 